# MPC para la verificación multiplicativa

En este notebook, vamos a explicar cómo funciona el protocolo que se permite verificar una tripleta multiplicativa a partir de otra. Usando el concepto de esquemas de compartición de secretos, una tripleta multiplicativa es una tripleta de "comparticiones" de la forma ($ [a], [b], [c] $) donde $a \cdot b = c$.

En este protocolo queremos demostrar que conocemos una tripleta multiplicativa ($ [x], [y], [z] $) a partir de otra ($ [a], [b], [c] $).

1. Las partes obtienen un valor aleatorio $ \epsilon \in \mathbb{F} $ (proporcionado por el verificador en el paradigma MPCitH).

2. Las partes calculan localmente $ [\alpha] = \epsilon [x] + [a] $ y $ [\beta] = [y] + [b] $.

3. Las partes transmiten $ [\alpha] $ y $ [\beta] $ para obtener $ \alpha $ y $ \beta $.

4. Las partes calculan localmente:

   $$
   [\nu] = \epsilon [z] - [c] + \alpha \cdot [b] + \beta \cdot [a] - \alpha \cdot \beta
   $$

5. Las partes transmiten $ [\nu] $ para obtener $ \nu $.

6. Las partes aceptan si $ \nu = 0 $ y rechazan en caso contrario.


En este caso es necesario considerar que se está construyendo una prueba correcta. Es necesario considerar el caso en el cual las tripletas multiplicativas no son correctas y cómo esto puede llegar a ser cierto.

Para esto, remitirse a los documentos donde se plantea el protocolo propuesto.

* A Framework for Constructing Fast MPC over Arithmetic Circuits with Malicious Adversaries and an Honest-Majority (https://dl.acm.org/doi/10.1145/3133956.3133999).

## Ejemplo

In [34]:
# Definir parámetros
F = GF(7919)  # Campo finito de tamaño primo
num_partes = 5  # Definir la cantidad de partes

# Paso 1: Obtener un valor aleatorio epsilon en el campo finito
epsilon = F.random_element()

# Paso 2: Generar valores secretos x, y, a, b
# Calcular z = x * y y c = a * b
x = F.random_element()
y = F.random_element()
z = x * y
a = F.random_element()
b = F.random_element()
c = a * b

# Función para generar comparticiones aditivas
def generar_comparticiones(secret, num_partes):
    comparticiones = [F.random_element() for _ in range(num_partes - 1)]
    suma_comparticiones = sum(comparticiones)
    ultima_comparticion = secret - suma_comparticiones
    comparticiones.append(ultima_comparticion)
    return comparticiones

# Función para reconstruir el secreto a partir de las comparticiones
def reconstruir_secreto(comparticiones):
    return sum(comparticiones)

# Función para calcular comparticiones de alpha
def calcular_comparticion_alpha(epsilon, inputs_por_parte):
    return [epsilon * parte[0] + parte[3] for parte in inputs_por_parte]

# Función para calcular comparticiones de beta
def calcular_comparticion_beta(inputs_por_parte):
    return [parte[1] + parte[4] for parte in inputs_por_parte]

# Función para reconstruir alpha
def reconstruir_alpha(comparticiones_alpha):
    return sum(comparticiones_alpha)

# Función para reconstruir beta
def reconstruir_beta(comparticiones_beta):
    return sum(comparticiones_beta)

# Función para reconstruir v
def reconstruir_v(comparticiones_v, alpha, beta):
    v = 0
    for i in range(len(comparticiones_v)):
        if i == 0:
            v += comparticiones_v[i] - alpha * beta
        else:
            v += comparticiones_v[i]
    return v

# Generar comparticiones aditivas para cada valor
comparticion_x = generar_comparticiones(x, num_partes)
comparticion_y = generar_comparticiones(y, num_partes)
comparticion_z = generar_comparticiones(z, num_partes)
comparticion_a = generar_comparticiones(a, num_partes)
comparticion_b = generar_comparticiones(b, num_partes)
comparticion_c = generar_comparticiones(c, num_partes)

# Crear lista de tuplas donde cada tupla representa los inputs de cada parte
inputs_por_parte = list(zip(comparticion_x, comparticion_y, comparticion_z, comparticion_a, comparticion_b, comparticion_c))

# Paso 3: Calcular localmente [alpha] y [beta] para cada parte
comparticion_alpha = calcular_comparticion_alpha(epsilon, inputs_por_parte)
comparticion_beta = calcular_comparticion_beta(inputs_por_parte)

# Paso 4: Transmitir [alpha] y [beta] para obtener alpha y beta
alpha = reconstruir_alpha(comparticion_alpha)
beta = reconstruir_beta(comparticion_beta)

# Paso 5: Calcular localmente [v] para cada parte
comparticion_v = [epsilon * parte[2] - parte[5] + alpha * parte[4] + beta * parte[3] for parte in inputs_por_parte]

# Paso 6: Transmitir [v] para obtener v
v = reconstruir_v(comparticion_v, alpha, beta)

# Paso 7: Verificar si v es igual a 0
if v == 0:
    print("Las partes aceptan: v = 0")
else:
    print("Las partes rechazan: v =", v)

Las partes aceptan: v = 0
